# Putting it all together (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
# Install required libraries for Transformers, datasets, and evaluation
!uv pip install datasets evaluate transformers[sentencepiece]

In [ ]:
# Basic tokenization - tokenizer handles the complete preprocessing pipeline
# This automatically adds special tokens ([CLS], [SEP]) and converts to IDs
# Returns a dictionary with input_ids, attention_mask, and token_type_ids
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)

In [ ]:
# Single sequence processing example
# Each call to the tokenizer processes the input completely
sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)

In [ ]:
# Batch processing - handle multiple sequences at once
# When sequences have different lengths, they need to be padded
# The tokenizer can handle batching automatically
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

model_inputs = tokenizer(sequences)

In [ ]:
# Padding strategies - control how sequences are padded in batches
# padding="longest": pad to the longest sequence in the current batch
# padding="max_length": pad to the model's maximum length (512 for BERT/DistilBERT)
# max_length parameter: specify custom maximum length

# Will pad the sequences up to the maximum sequence length
model_inputs = tokenizer(sequences, padding="longest")

# Will pad the sequences up to the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, padding="max_length")

# Will pad the sequences up to the specified max length
model_inputs = tokenizer(sequences, padding="max_length", max_length=8)

In [ ]:
# Truncation strategies - handle sequences that are too long
# truncation=True: cut sequences longer than the model's max length
# max_length + truncation: specify custom length limit
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# Will truncate the sequences that are longer than the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, truncation=True)

# Will truncate the sequences that are longer than the specified max length
model_inputs = tokenizer(sequences, max_length=8, truncation=True)

In [ ]:
# Return tensor types - specify the format for model consumption
# return_tensors parameter controls the output format:
# "pt": PyTorch tensors (for PyTorch models)
# "tf": TensorFlow tensors (for TensorFlow/Keras models)  
# "np": NumPy arrays (for other frameworks or manual processing)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# Returns PyTorch tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="pt")

# Returns TensorFlow tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")

# Returns NumPy arrays
model_inputs = tokenizer(sequences, padding=True, return_tensors="np")

In [ ]:
# Comparing tokenizer() vs manual tokenization
# Notice the difference: tokenizer() adds special tokens automatically
# Manual tokenization requires you to handle special tokens yourself
sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"])  # Includes [CLS] (101) and [SEP] (102) tokens

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)  # Raw tokens without special tokens

In [ ]:
# Decode back to text to see the difference
# The tokenizer() output includes special tokens in the decoded text
# Manual tokenization output doesn't include special tokens
print(tokenizer.decode(model_inputs["input_ids"]))  # With [CLS] and [SEP]
print(tokenizer.decode(ids))  # Without special tokens

In [ ]:
# Complete pipeline: tokenization + model inference
# This combines everything we've learned:
# 1. Load tokenizer and model
# 2. Process multiple sequences with padding and truncation
# 3. Convert to tensors for PyTorch
# 4. Pass through model using ** to unpack the dictionary
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)  # ** unpacks the dictionary (input_ids, attention_mask, etc.)